In [1]:
import glob, os, gc
import os.path
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from __future__ import division
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn.decomposition import (PCA, RandomizedPCA)
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from IPython.display import Image
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import pybedtools
import pysam
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
basepath = "/home/ubuntu/efs/Common_Experiment/final" 
narrowpeak = "-ready_peaks.narrowPeak"
peaks_merged = "_peaksMerged.txt"
peaks_merged_bed = "_peaksMerged.bed"
peaks_correct_bed = "_peaksCorrect.bed"
peakregions_sml = ".peakregions_sml"
peakregions_lrg = ".peakregions_lrg"
peaks_correct_data = "_peaksCorrect.data"
peaks_L1HS_bedgraph = "_peaks_L1HS_mapped.bedgraph"
loci_sml = ".loci_sml"
loci_lrg = ".loci_lrg"
overlap = "_overlap_"
overlap_sml = "_overlap_sml_"
overlap_lrg = "_overlap_lrg_"
L1HS_bam = "-L1HS_mapped.bam"
L1HS_bam_bai = "-L1HS_mapped.bam.bai"
bam = "-ready.bam"
igv = "-igv.xml"
bed = ".bed"
## rmask Paths 
L1HS = "/home/ubuntu/efs/SLAV_Data/rmask_L1HS_Final.bed"
L1PA2345 = "/home/ubuntu/efs/SLAV_Data/rmask_L1PA2345_Final.bed"
L1_Other = "/home/ubuntu/efs/SLAV_Data/rmask_L1_Other_Final.bed"
##IGV Template
IGV = "/home/ubuntu/efs/SLAV_Data/igv-template4.xml"

##SLAV Data
Bulk_1571_Cerebellum = "1571_cereb_BT_40_L3"
Bulk_1571_Hippocampus = "1571_hippo_BT_41_L3"
Bulk_1846_Cerebellum = "1846_cereb_BT_13_L3"
Bulk_1846_Cortex = "1846_cortex_BT_71_L3"
Bulk_1846_Hippocampus = "1846_hippo_BT_19_L3"
Bulk_1846_Liver = "1846_liver_BT_22_L3"
Bulk_5125_Cortex = "5125_cortex_BT_122_L3"
Bulk_5125_Hippocampus = "5125_hippo_BT_139_L3"
Bulk_5125_Liver = "5125_liver_BT_164_L3"
#
SC_1571_Hippo = ["1571_hippo_SC_43_L3","1571_hippo_SC_45_L3","1571_hippo_SC_46_L3","1571_hippo_SC_47_L3","1571_hippo_SC_48_L3","1571_hippo_SC_50_L3","1571_hippo_SC_51_L3","1571_hippo_SC_52_L3","1571_hippo_SC_53_L3","1571_hippo_SC_55_L3","1571_hippo_SC_56_L3","1571_hippo_SC_57_L3","1571_hippo_SC_58_L3","1571_hippo_SC_59_L3","1571_hippo_SC_61_L3","1571_hippo_SC_62_L3","1571_hippo_SC_63_L3","1571_hippo_SC_64_L3"]
SC_1846_Cortex = ["1846_cortex_SC_72_L3","1846_cortex_SC_73_L3","1846_cortex_SC_74_L3","1846_cortex_SC_75_L3","1846_cortex_SC_78_L3","1846_cortex_SC_79_L3","1846_cortex_SC_80_L3","1846_cortex_SC_81_L3","1846_cortex_SC_82_L3","1846_cortex_SC_83_L3","1846_cortex_SC_84_L3","1846_cortex_SC_85_L3","1846_cortex_SC_86_L3"]
SC_1846_Hippo = ["1846_hippo_SC_100_L3","1846_hippo_SC_101_L3","1846_hippo_SC_102_L3","1846_hippo_SC_103_L3","1846_hippo_SC_104_L3","1846_hippo_SC_105_L3","1846_hippo_SC_106_L3","1846_hippo_SC_107_L3","1846_hippo_SC_108_L3","1846_hippo_SC_109_L3","1846_hippo_SC_110_L3","1846_hippo_SC_111_L3","1846_hippo_SC_112_L3","1846_hippo_SC_113_L3","1846_hippo_SC_88_L3","1846_hippo_SC_89_L3","1846_hippo_SC_90_L3","1846_hippo_SC_91_L3","1846_hippo_SC_92_L3","1846_hippo_SC_93_L3","1846_hippo_SC_94_L3","1846_hippo_SC_95_L3","1846_hippo_SC_99_L3"]
SC_5125_Cortex = ["5125_cortex_SC_125_L3","5125_cortex_SC_126_L3","5125_cortex_SC_127_L3","5125_cortex_SC_128_L3","5125_cortex_SC_129_L3","5125_cortex_SC_130_L3","5125_cortex_SC_131_L3","5125_cortex_SC_132_L3","5125_cortex_SC_133_L3","5125_cortex_SC_134_L3","5125_cortex_SC_135_L3","5125_cortex_SC_136_L3","5125_cortex_SC_138_L3"]
SC_5125_Hippo = ["5125_hippo_SC_140_L3","5125_hippo_SC_141_L3","5125_hippo_SC_142_L3","5125_hippo_SC_143_L3","5125_hippo_SC_144_L3","5125_hippo_SC_145_L3","5125_hippo_SC_147_L3","5125_hippo_SC_149_L3","5125_hippo_SC_150_L3","5125_hippo_SC_151_L3","5125_hippo_SC_152_L3","5125_hippo_SC_153_L3","5125_hippo_SC_154_L3","5125_hippo_SC_155_L3","5125_hippo_SC_156_L3","5125_hippo_SC_157_L3","5125_hippo_SC_158_L3","5125_hippo_SC_159_L3","5125_hippo_SC_160_L3","5125_hippo_SC_161_L3","5125_hippo_SC_162_L3","5125_hippo_SC_163_L3"]

##Common Data
Bulk_Brain_Common = "04132016_mw_Bulk_cor"
Bulk_Fibro_Common = "05252016_mw_Bulk_fib"
SC_MDA_Common = ["04132016_mw_1571_SC_B4_S48","04132016_mw_1571_SC_D4_S49","04132016_mw_L1B1_SC_A2_S43","04132016_mw_L1B1_SC_C1_S45","04132016_mw_L1B1_SC_C2_S46","04132016_mw_L1B1_SC_D2_S50","04132016_mw_L1B1_SC_E2_S51","04132016_mw_L1B1_SC_E3_S52","04132016_mw_L1B1_SC_E3_S52","04132016_mw_L1B1_SC_F2_S53","04132016_mw_L1B1_SC_G1_S54","04132016_mw_L1B1_SC_H1_S55","05252016_mw_L1B1_SC_B4_S47"] 
SC_MALBAC_Common = ["2122_S1","2178_S2","2179_S3","2180_S4","2184_S5","2186_S6","2187_S7","2188_S8","2193_S9","2196_S10","2197_S11","2198_S12","2261_S13","2263_S14","2264_S15","2265_S16"] 



Data_Sets = []
#Data_Sets.append([SC_1571_Hippo,Bulk_1571_Hippocampus,Bulk_1571_Cerebellum])
#Data_Sets.append([SC_1846_Cortex,Bulk_1846_Cortex,Bulk_1846_Liver])
#Data_Sets.append([SC_1846_Hippo,Bulk_1846_Hippocampus,Bulk_1846_Liver])
#Data_Sets.append([SC_5125_Cortex,Bulk_5125_Cortex,Bulk_5125_Liver])
#Data_Sets.append([SC_5125_Hippo,Bulk_5125_Hippocampus,Bulk_5125_Liver])

Data_Sets = []
Data_Sets.append([SC_MDA_Common,Bulk_Brain_Common,Bulk_Fibro_Common])
Data_Sets.append([SC_MALBAC_Common,Bulk_Brain_Common,Bulk_Fibro_Common])

/home/ubuntu/anaconda3/envs/python2/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python2/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python2/lib/python2.7/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [6]:
## soft call peaks in bulk tissues
for dset in Data_Sets:
    tissue = dset[1]
    os.chdir(os.path.join(basepath, tissue))
    if not (glob.glob(os.path.join(basepath, tissue, "peaks.txt"))):
        print tissue
        name = os.path.join(basepath, tissue, tissue)
        bamfile = os.path.join(basepath, tissue, tissue+bam)
        #p1 = Popen(['makeTagDirectory', '.', bamfile, '-format', 'sam', '-keepAll', '-single'])
        #p1.wait()
        p2a = Popen(['findPeaks', '.', '-o', 'peaks.txt', '-style', 'dnase', '-F', '0', '-L', '0', '-C', '0', '-tagThreshold', '5'])
        p2a.wait()
        p4 = Popen(['pos2bed.pl', '-bed', os.path.join(basepath, tissue, "peaks.txt")])
        p4.wait()
    tissue = dset[2]
    os.chdir(os.path.join(basepath, tissue))
    if not (glob.glob(os.path.join(basepath, tissue, "peaks.txt"))):
        print tissue
        name = os.path.join(basepath, tissue, tissue)
        bamfile = os.path.join(basepath, tissue, tissue+bam)
        #p1 = Popen(['makeTagDirectory', '.', bamfile, '-format', 'sam', '-keepAll', '-single'])
        #p1.wait()
        p2a = Popen(['findPeaks', '.', '-o', 'peaks.txt', '-style', 'dnase', '-F', '0', '-L', '0', '-C', '0', '-tagThreshold', '5'])
        p2a.wait()
        p4 = Popen(['pos2bed.pl', '-bed', os.path.join(basepath, tissue, "peaks.txt")])
        p4.wait()                

05252016_mw_Bulk_fib


In [8]:
## soft call peaks in single cells
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, cell))
        name = os.path.join(basepath, cell, cell)
        bamfile = os.path.join(basepath, cell, cell+bam)
        #p1 = Popen(['/home/preed/homer/bin/makeTagDirectory', '.', bamfile, '-format', 'sam', '-keepAll', '-single'])
        #p1.wait()
        p2a = Popen(['findPeaks', '.', '-o', 'peaks.txt', '-style', 'dnase', '-F', '0', '-L', '0', '-C', '0', '-tagThreshold', '5'])
        p2a.wait()
        p4 = Popen(['pos2bed.pl', '-bed', os.path.join(basepath, cell, "peaks.txt")])
        p4.wait()
        p5 = Popen(['mergePeaks', '-code', '-prefix', cell, os.path.join(basepath, cell, "peaks.txt"), os.path.join(basepath, dset[1], "peaks.txt"), os.path.join(basepath,  dset[2], "peaks.txt")])
        p5.wait()
        #myoutput = open(os.path.join(basepath, cell, cell + peaks_merged), 'w')
        #p5 = Popen(['/home/preed/homer/bin/mergePeaks', os.path.join(basepath, cell, "peaks.txt"), os.path.join(basepath, dset[1], "peaks.txt"), os.path.join(basepath,  dset[2], "peaks.txt")], stdout=myoutput)
        #p5.wait()
        myoutput = open(os.path.join(basepath, cell, cell + peaks_merged), 'w')
        p6 = Popen(['mergePeaks', os.path.join(basepath,  cell, cell+"_100"),os.path.join(basepath,  cell, cell+"_110"),os.path.join(basepath,  cell, cell+"_111")], stdout=myoutput)
        p6.wait()
        p7 = Popen(['pos2bed.pl', os.path.join(basepath,  cell, cell + peaks_merged), '-o', os.path.join(basepath,  cell, cell+peaks_merged_bed)])
        p7.wait() 

04132016_mw_1571_SC_B4_S48
04132016_mw_1571_SC_D4_S49
04132016_mw_L1B1_SC_A2_S43
04132016_mw_L1B1_SC_C1_S45
04132016_mw_L1B1_SC_C2_S46
04132016_mw_L1B1_SC_D2_S50
04132016_mw_L1B1_SC_E2_S51
04132016_mw_L1B1_SC_E3_S52
04132016_mw_L1B1_SC_E3_S52
04132016_mw_L1B1_SC_F2_S53
04132016_mw_L1B1_SC_G1_S54
04132016_mw_L1B1_SC_H1_S55
05252016_mw_L1B1_SC_B4_S47
2122_S1
2178_S2
2179_S3
2180_S4
2184_S5
2186_S6
2187_S7
2188_S8
2193_S9
2196_S10
2197_S11
2198_S12
2261_S13
2263_S14
2264_S15
2265_S16


In [2]:
##make XML file for each cell to use with igv_plotter
for dset in Data_Sets:
    for cell in dset[0]:
        print cell
        tree = ET.parse(IGV)
        root = tree.getroot()
        root[0][0].set('path', os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)) #L1HS bedgraph
        root[0][1].set('path', os.path.join(basepath,  cell, cell + bam)) #SC Path
        root[0][3].set('path', os.path.join(basepath,  dset[1], dset[1] + bam)) #Bulk Brain Path
        root[0][6].set('path', os.path.join(basepath,  dset[2], dset[2] + bam)) #Bulk Fib Path
        root[1][0].set('id', os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)) #L1HS bedgraph
        root[2][0].set('id', os.path.join(basepath,  cell, cell + bam)) #SC Path
        root[3][0].set('id', os.path.join(basepath,  dset[1], dset[1] + bam)) #Bulk Brain path
        root[4][0].set('id', os.path.join(basepath,  dset[2], dset[2] + bam)) #Bulk Fib Path
        tree.write(os.path.join(basepath,  cell, cell + igv))



        myinput = open(os.path.join(basepath,  cell, cell + peaks_merged_bed))
        myoutput = open(os.path.join(basepath,  cell, cell + peaks_correct_bed), 'w')
        proc3 = Popen(['grep', '-E', '^(1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|X|Y)'], stdin=myinput, stdout=myoutput)
        proc3.wait()    

        myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
        myoutput1 = os.path.join(basepath,  cell, cell + peakregions_sml)
        myoutput2 = os.path.join(basepath,  cell, cell + peakregions_lrg)
        with open(myoutput1, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for string in data:
                    line = string.split('\t')
                    pos1 = int(line[1])
                    pos2 = int(line[2])                  
                    center = int((pos1 + pos2)/2)
                    pad = 1000
                    start = center - pad
                    end = center + pad
                    row = [line[0], str(start), str(end)]
                    outfile.write('\t'.join(row) + '\n')
        outfile.close()
        infile.close()   
        with open(myoutput2, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for string in data:
                    line = string.split('\t')
                    pos1 = int(line[1])
                    pos2 = int(line[2])                  
                    center = int((pos1 + pos2)/2)
                    pad = 10000
                    newstart = center - pad
                    newend = center + pad
                    row = [line[0], str(newstart), str(newend)]
                    outfile.write('\t'.join(row) + '\n')
        outfile.close()
        infile.close()
 
        #make and define L1HS sub sam file here and L1HS read names list
        
               
        sc_file = pysam.AlignmentFile(os.path.join(basepath,  cell, cell + bam), "rb")
        bb_file = pysam.AlignmentFile(os.path.join(basepath,  dset[1], dset[1] + bam), "rb")
        bf_file = pysam.AlignmentFile(os.path.join(basepath,  dset[2], dset[2] + bam), "rb")
        

        myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
        myoutput = os.path.join(basepath,  cell, cell + peaks_correct_data)
        
        with open(myoutput, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    sc_iter = sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                    bb_iter = bb_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                    bf_iter = bf_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                    sc_i = 0
                    bb_i = 0
                    bf_i = 0
                    for x in sc_iter: sc_i+=1
                    for y in bb_iter: bb_i+=1
                    for z in bf_iter: bf_i+=1
                    sc_count = 1 if sc_i > 0 else 0
                    bb_count = 1 if bb_i > 0 else 0 
                    bf_count = 1 if bf_i > 0 else 0 
                    row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2])]  
                    outfile.write('\t'.join(row) +'\t'+str(sc_count)+str(bb_count)+str(bf_count)+'\n')
        outfile.close()
        infile.close()
        
        myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
        myoutput = open(os.path.join(basepath, cell, cell + L1HS_bam), 'w')
        myoutput2 = os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)
        p1 = Popen(['samtools', 'view', '-hb', '-@', '1', '-L', L1HS, os.path.join(basepath, cell, cell + bam)], stdout=myoutput)
        p1.wait()
        myoutput.close()
        p2 = Popen(['samtools', 'index', os.path.join(basepath, cell, cell + L1HS_bam)])
        p2.wait()
        L1HS_file = pysam.AlignmentFile(os.path.join(basepath, cell, cell + L1HS_bam), 'rb')
        L1HS_read_names = []
        for read in L1HS_file.fetch(): 
            read_name = str(read).split('\t')[0]#.split('.')[1])
            L1HS_read_names.append(read_name)
        with open(myoutput2, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    read_names = ["empty"]
                    for read in sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2])):
                        read_name = str(read).split('\t')[0]#.split('.')[1])
                        read_names.append(read_name)
                    a = len(read_names)
                    L1HS_overlap = set(read_names) & set(L1HS_read_names)
                    b = len(L1HS_overlap)
                    percent = int((b/a)*100)
                    row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2]), str(percent)]  
                    outfile.write('\t'.join(row)+'\n')
        outfile.close()
        infile.close()
        
        filelist =[os.path.join(basepath,  cell, "peaks.bed"),os.path.join(basepath,  dset[1], "peaks.bed"),os.path.join(basepath,  dset[2], "peaks.bed"),L1HS,L1PA2345,L1_Other]
        a = pybedtools.BedTool(os.path.join(basepath, cell, cell + peaks_correct_bed)) ##mergedpeaks2 overlap with loci window or with peak location???
        count = 0
        for fname in filelist:
            b = pybedtools.BedTool(fname)
            a_and_b = a.intersect(b, c=True)
            myoutput = os.path.join(basepath,  cell, cell + overlap + str(count))
            count +=1
            a_and_b.saveas(myoutput)
            myinput = open(myoutput)
            newoutput = open(myoutput+"_binary", 'w')
            #overlap_append
            awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($7 ~ "^[1-9]*$") $7 = "1"; else $7 = $7; }; 1"""
            proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
            proc.wait()
            newoutput.flush()
#         peaks = open(os.path.join(basepath, cell, cell + peaks_correct_bed))
#         peaks_sorted = open(os.path.join(basepath, cell, cell + peaks_correct_bed_sorted), 'w')
#         proc = Popen(["sort", "-k1,1", "-k2,2n"], stdin=peaks, stdout=peaks_sorted)  
#         proc.wait()
#         l1_input = open(os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph))
#         l1_sorted = open(os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph_sorted), 'w')
#         proc = Popen(["sort", "-k1,1", "-k2,2n"], stdin=l1_input, stdout=l1_sorted)  
#         proc.wait()
#         a_sorted = pybedtools.BedTool(os.path.join(basepath, cell, cell + peaks_correct_bed_sorted)) ##mergedpeaks2 overlap with loci window or with peak location???
#         l1 = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph_sorted))
#         a_and_l1 = a_sorted.map(l1, c=4, o='sum')
#         myoutput_l1 = os.path.join(basepath,  cell, cell + overlap + "L1")
#         a_and_l1.saveas(myoutput_l1)
        
        a_sml = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peakregions_sml)) ##mergedpeaks2 overlap with loci window or with peak location???
        count = 0
        for fname in filelist:
            b = pybedtools.BedTool(fname)
            a_and_b = a_sml.intersect(b, c=True)
            myoutput = os.path.join(basepath,  cell, cell + overlap_sml + str(count))
            count +=1
            a_and_b.saveas(myoutput)
            myinput = open(myoutput)
            newoutput = open(myoutput+"_binary", 'w')
            #overlap_append
            awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
            proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
            proc.wait()
            newoutput.flush()

        a_lrg = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peakregions_lrg)) ##mergedpeaks2 overlap with loci window or with peak location???
        count = 0
        for fname in filelist:
            b = pybedtools.BedTool(fname)
            a_and_b = a_lrg.intersect(b, c=True)
            myoutput = os.path.join(basepath,  cell, cell + overlap_lrg + str(count))
            count +=1
            a_and_b.saveas(myoutput)
            myinput = open(myoutput)
            newoutput = open(myoutput+"_binary", 'w')
            #overlap_append
            awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
            proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
            proc.wait()
            newoutput.flush()        

        myinput_sml = os.path.join(basepath,  cell, cell + peakregions_sml)
        myoutput_sml = os.path.join(basepath,  cell, cell + loci_sml)
        with open(myoutput_sml, 'w') as outfile:
            with open(myinput_sml, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
                    outfile.write("".join(row)+'\n')

        myinput_lrg = os.path.join(basepath,  cell, cell + peakregions_lrg)
        myoutput_lrg = os.path.join(basepath,  cell, cell + loci_lrg)
        with open(myoutput_lrg, 'w') as outfile:
            with open(myinput_lrg, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
                    outfile.write("".join(row)+'\n')    

        Popen(['split', '-l', '100', '-d', os.path.join(basepath,  cell, cell + loci_sml), os.path.join(basepath,  cell, cell + ".split_loci_sml_")]).wait()
        Popen(['split', '-l', '100', '-d', os.path.join(basepath,  cell, cell + loci_lrg), os.path.join(basepath,  cell, cell + ".split_loci_lrg_")]).wait()

04132016_mw_1571_SC_B4_S48
04132016_mw_1571_SC_D4_S49
04132016_mw_L1B1_SC_A2_S43


KeyboardInterrupt: 

In [4]:

#generate images at each peak position small
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, cell))    
        locifile = os.path.join(basepath, cell, cell + loci_sml)
        worklist = glob.glob("*.split_loci_sml_*")
        batchsize = 10
        print len(worklist)
        for i in xrange(0, len(worklist), batchsize):
            batch = worklist[i:i+batchsize]
            print i
            index = 1
            procs = []
            for file in batch:
                print file
                with open(os.path.join(basepath, cell, file)) as f0:
                    first = f0.readline()# Read the first line.
                    for last in f0: pass
                    firstpic = cell+"_sml"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                    lastpic = cell+"_sml"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                    if not (glob.glob(os.path.join(basepath, cell, firstpic)) or glob.glob(os.path.join(basepath, cell, lastpic))): 
                        p = Popen(['igv_plotter', '-o', cell+"_sml_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/home/ubuntu/efs/SLAV_Data/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, cell, cell + igv)])
                        procs.append(p)
            for pp in procs:
                pp.wait()
                #wait_timeout(pp,300)

04132016_mw_1571_SC_B4_S48
0
04132016_mw_1571_SC_D4_S49
0
04132016_mw_L1B1_SC_A2_S43
0
04132016_mw_L1B1_SC_C1_S45
0
04132016_mw_L1B1_SC_C2_S46
0
04132016_mw_L1B1_SC_D2_S50
0
04132016_mw_L1B1_SC_E2_S51
0
04132016_mw_L1B1_SC_E3_S52
0
04132016_mw_L1B1_SC_E3_S52
0
04132016_mw_L1B1_SC_F2_S53
0
04132016_mw_L1B1_SC_G1_S54
0
04132016_mw_L1B1_SC_H1_S55
0
05252016_mw_L1B1_SC_B4_S47
0
2122_S1
0
2178_S2
0
2179_S3
0
2180_S4
0
2184_S5
0
2186_S6
0
2187_S7
0
2188_S8
0
2193_S9
0
2196_S10
0
2197_S11
0
2198_S12
0
2261_S13
0
2263_S14
0
2264_S15
0
2265_S16
0


In [ ]:
#generate images at each peak position lrg
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, diry, cell))    
        locifile = os.path.join(basepath, diry, cell, cell + loci_lrg)
        worklist = glob.glob("*.split_loci_lrg_*")
        batchsize = 10
        print len(worklist)
        for i in xrange(0, len(worklist), batchsize):
            batch = worklist[i:i+batchsize]
            print i
            index = 1
            procs = []
            for file in batch:
                print file
                with open(os.path.join(basepath, diry, cell, file)) as f0:
                    first = f0.readline()# Read the first line.
                    for last in f0: pass
                    firstpic = cell+"_lrg"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                    lastpic = cell+"_lrg"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                    if not (glob.glob(os.path.join(basepath, diry, cell, firstpic)) or glob.glob(os.path.join(basepath, diry, cell, lastpic))): 
                        p = Popen(['igv_plotter', '-o', cell+"_lrg_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/home/ubuntu/efs/SLAV_Data/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, diry, cell, cell + igv)])
                        procs.append(p)
            for pp in procs:
                pp.wait()
                #wait_timeout(pp,300)

In [ ]:
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
#         for file in glob.glob("*.png"):
#             if "mod" not in file:
#                 path = os.path.splitext(file)[0]
#                 basename = os.path.basename(path)
#                 outfile1 = basename + "_mod.png"
#                 if not os.path.isfile(os.path.join(basepath,cell,outfile1)):               
#                     img = Image.open(file)
#                     pixelMap = img.load()
#                     img2 = Image.new(img.mode, img.size)
#                     pixelsNew = img2.load()
#                     for i in range(img2.size[0]):
#                         for j in range(img2.size[1]):
#                             if 250 in pixelMap[i,j]:
#                                 pixelMap[i,j] = (0,0,0,0)
#                             else:
#                                 pixelsNew[i,j] = pixelMap[i,j]
#                 img2.crop((160,130,img.size[0],img.size[1])).resize((256,256)).save(outfile1)

        mergedpeak_data = os.path.join(basepath, cell, cell + peaks_correct_data)
        regions_sml = os.path.join(basepath, cell, cell + peakregions_sml)
        regions_lrg = os.path.join(basepath, cell, cell + peakregions_lrg)

        count=1
        with open(mergedpeak_data) as r0,open(regions_sml) as r_sml,open(regions_lrg) as r_lrg:
            Files= {}
            for rr0,rr_sml,rr_lrg in zip(r0,r_sml,r_lrg):
                line = rr0.strip().split('\t')[0]+"\t"+rr0.strip().split('\t')[1]+"\t"+rr0.strip().split('\t')[2]+"\t"+cell+"_sml-"+rr_sml.strip().split('\t')[0]+"_"+rr_sml.strip().split('\t')[1]+"_"+rr_sml.strip().split('\t')[2]+"_blk.png"+"\t"+cell+"_lrg-"+rr_lrg.strip().split('\t')[0]+"_"+rr_lrg.strip().split('\t')[1]+"_"+rr_lrg.strip().split('\t')[2]+"_blk.png"+"\t"+rr0.strip().split('\t')[3]
                Files[str(count)] = line
                count+=1

        a = os.path.join(basepath, cell, cell+"_overlap_0_binary")
        b = os.path.join(basepath, cell, cell+"_overlap_1_binary")
        c = os.path.join(basepath, cell, cell+"_overlap_2_binary")
        d = os.path.join(basepath, cell, cell+"_overlap_3_binary")
        e = os.path.join(basepath, cell, cell+"_overlap_4_binary")
        f = os.path.join(basepath, cell, cell+"_overlap_5_binary")
        g = os.path.join(basepath, cell, cell+ peaks_L1HS_bedgraph)
        count=1
        with open(a) as f1,open(b) as f2,open(c) as f3,open(d) as f4,open(e) as f5,open(f) as f6,open(g) as f7:
            Peaks = {}
            for aa,bb,cc,dd,ee,ff,gg in zip(f1,f2,f3,f4,f5,f6,f7):
                line = aa.strip().split('\t')[6]+bb.strip().split('\t')[6]+cc.strip().split('\t')[6]+"\t"+dd.strip().split('\t')[6]+ee.strip().split('\t')[6]+ff.strip().split('\t')[6]+"\t"+gg.strip().split('\t')[3]
                Peaks[str(count)] = line
                count+=1

        a_sml = os.path.join(basepath, cell, cell+"_overlap_sml_0_binary")
        b_sml = os.path.join(basepath, cell, cell+"_overlap_sml_1_binary")
        c_sml = os.path.join(basepath, cell, cell+"_overlap_sml_2_binary")
        d_sml = os.path.join(basepath, cell, cell+"_overlap_sml_3_binary")
        e_sml = os.path.join(basepath, cell, cell+"_overlap_sml_4_binary")
        f_sml = os.path.join(basepath, cell, cell+"_overlap_sml_5_binary")
        count=1
        with open(a_sml) as f1,open(b_sml) as f2,open(c_sml) as f3,open(d_sml) as f4,open(e_sml) as f5,open(f_sml) as f6:
            Small = {}
            for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
                line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
                Small[str(count)] = line
                count+=1

        a_lrg = os.path.join(basepath, cell, cell+"_overlap_lrg_0_binary")
        b_lrg = os.path.join(basepath, cell, cell+"_overlap_lrg_1_binary")
        c_lrg = os.path.join(basepath, cell, cell+"_overlap_lrg_2_binary")
        d_lrg = os.path.join(basepath, cell, cell+"_overlap_lrg_3_binary")
        e_lrg = os.path.join(basepath, cell, cell+"_overlap_lrg_4_binary")
        f_lrg = os.path.join(basepath, cell, cell+"_overlap_lrg_5_binary")
        count=1
        with open(a_lrg) as f1,open(b_lrg) as f2,open(c_lrg) as f3,open(d_lrg) as f4,open(e_lrg) as f5,open(f_lrg) as f6:
            Large = {}
            for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
                line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
                Large[str(count)] = line
                count+=1

        with open(os.path.join(basepath, cell, cell+"_Input_metadata.txt"),"w") as f8:
            for key in Files:
                encoding = Files[key]+":"+Peaks[key].strip().split('\t')[0]+":"+Peaks[key].strip().split('\t')[1]+":"+Small[key].strip().split('\t')[0]+":"+Small[key].strip().split('\t')[1]+":"+Large[key].strip().split('\t')[0]+":"+Large[key].strip().split('\t')[1]+":"+Peaks[key].strip().split('\t')[2]+"\n"            #for (Fi, Fj),(Pi,Pj),(Si,Sj),(Li,Lj) in zip(Files.items(),Peaks.items(),Small.items(),Large.items()):
                f8.write(encoding)

In [ ]:
for dset in Data_Sets:
    for cell in dset[0]:
        print cell
        with open(os.path.join(basepath, cell, cell+"_Input_metadata.txt")) as f:
            for line in csv.reader(f, delimiter="\t"):
                if os.path.isfile(os.path.join(basepath, cell, line[3])):
                    #print line[3]
                    filename = line[3]
                    readclass = line[5].split(":")[0]
                    peakclass_peak = line[5].split(":")[1]
                    L1_class_peak = line[5].split(":")[2]
                    peakclass_sml = line[5].split(":")[3]
                    L1_class_sml = line[5].split(":")[4]
                    peakclass_lrg = line[5].split(":")[5]
                    L1_class_lrg = line[5].split(":")[6]
                    L1_Percent = line[5].split(":")[7]
                    if 0 <= int(L1_Percent) <= 25:
                        class_sml = [readclass,peakclass_peak,L1_class_peak,peakclass_sml,L1_class_sml,peakclass_lrg,L1_class_lrg,"low"]
                    elif 25 < int(L1_Percent) < 75:
                        class_sml = [readclass,peakclass_peak,L1_class_peak,peakclass_sml,L1_class_sml,peakclass_lrg,L1_class_lrg,"med"]
                    elif 75 <= int(L1_Percent):
                        class_sml = [readclass,peakclass_peak,L1_class_peak,peakclass_sml,L1_class_sml,peakclass_lrg,L1_class_lrg,"high"]    
                    dst = "".join(class_sml)
                    if not os.path.exists(os.path.join(basepath, "BLK", dst)):
                        os.makedirs(os.path.join(basepath, "BLK", dst))
                    if not os.path.isfile(os.path.join(basepath, "BLK", dst,filename)):
                        shutil.move(os.path.join(basepath, cell, filename), os.path.join(basepath, "BLK", dst))